## Calculating S/N Estimates

- get stellar and planetary parameters from exoplanet archive
- get magnitudes and coords from simbad

In [6]:
from chromatic import * 

In [7]:
Tplanet = 600 * u.K # could recalculate this
Rplanet = 4.2*u.Rearth
Tstar = 3600*u.K
Rstar = 0.797*u.Rsun

## Spectroscopic Rotation Model for AU Mic b eclipse

- What is AU Mic's variability in that window of time
- Calculate it from the LCO bandpasses through the MIRI bandpass. We want to know wavelength-dependent variability in MIRI's bandpass but it needs to be consistent with the variability of 7-8% in 0.45-0.65 micron and down to ~3-6% in 0.7-0.8 micron

In [10]:
tau_obs = 10 *u.hr
T_rot = 4.863*u.day
T_orb = 8.463*u.day
Teff = 3700 * u.K
Tspot = 3000 * u.K # Herbst+ 2021 estimate, ~500-600K uncertainty
T_0 = 0.0 #planet mid-transit or stellar phase

In [20]:
! pip install speclite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:0000:0100:01


In [24]:
from batman import TransitParams
from fleck import Star, generate_spots
import numpy as np
import astropy.units as u
import astropy.constants as con
import emcee
import corner
import speclite as speclite; from speclite import filters
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from chromatic import get_phoenix_photons
from scipy.optimize import minimize
from scipy.optimize import curve_fit


params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
# plt.style.use('seaborn-deep')

In [ ]:
# Parameters for the planet
planet = TransitParams()
planet.per = 8.463
planet.a = 19.1#float(19.1*u.AU / u.R_sun)
planet.rp = 0.0512
planet.w = 90
planet.ecc = 0.04
planet.inc = 89.5
planet.t0 = 0
planet.limb_dark = 'quadratic'
planet.u = [0.3, 0.3]

# Parameters for normalizing the light curve
noplanet = planet
noplanet.rp = 0.0

# we need to load in different phoenix spectra for the appropriate wavelength coverage
- models from here: https://www.stsci.edu/hst/instrumentation/reference-data-for-calibration-and-tools/astronomical-catalogs/phoenix-models-available-in-synphothttps://www.stsci.edu/hst/instrumentation/reference-data-for-calibration-and-tools/astronomical-catalogs/phoenix-models-available-in-synphot

In [ ]:
from astropy.io import fits

spot_model = fits.open('/Users/wiwa8630/data/cool_phoenix_models/phoenixm00_2700.fits')
phot_model = fits.open('/Users/wiwa8630/data/cool_phoenix_models/phoenixm00_3900.fits')
spotspec_data = spot_model[1].data
photspec_data = phot_model[1].data
w = spotspec_data['WAVELENGTH']
flam_spot = spotspec_data['g45']
flam_phot = photspec_data['g45']

In [ ]:
start_index = 30012 # at 0.5 micron
end_index = 77506 # at 14 micron

#truncate the phoenix arrays
_w = w[start_index:end_index]/1e4
_spotspec = flam_spot[start_index:end_index]
_photspec = flam_phot[start_index:end_index]

plt.figure(figsize=(4,3))
plt.plot(_w,_spotspec)
plt.plot(_w,_photspec)
plt.xlabel('micron')
plt.ylabel(r'$erg~cm^{−2}~s^{−1}~\AA^{−1}$')
#plt.xlim(0.5,14)
#plt.ylim(1, None)
plt.yscale('log')
# plt.xscale('log')

our_wavelengths = np.array([0.48,0.63,0.77, 6, 8, 10, 12, 14])

spotspec = bintogrid(_w,_spotspec,newx=our_wavelengths,
                                      visualize=False)['y']
photspec = bintogrid(_w,_photspec,newx=our_wavelengths,
                                      visualize=False)['y']

plt.figure()
plt.plot(our_wavelengths,photspec)
plt.plot(our_wavelengths,spotspec)

contrasts = np.array(spotspec/photspec)
times = np.linspace(0, 5.0, 1000)*u.day
w = our_wavelengths*u.micron
N_times = len(times)
N_wavelengths = len(w)
lcs = np.ones(shape = (N_wavelengths, N_times))

cmap = cm.get_cmap('rainbow', len(contrasts))
colormap = cmap(np.linspace(0, 1, len(contrasts)))
u1 = np.linspace(0.5,0.1,N_wavelengths)
u2 = np.linspace(0.5,0.1,N_wavelengths)

In [ ]:
example_lons = [[ 23.22304363],
 [174.51727231],
 [237.54570828],
 [347.09567474],
 [261.8511381 ],
 [  1.45491167],
 [302.98117039],
 [ 72.60065268],
 [197.14034981]]
example_lats = [[-18.38038867],
 [-59.21172105],
 [ 17.83574762],
 [-53.84447223],
 [-41.17001579],
 [ -9.47024332],
 [-49.23752712],
 [-71.53816482],
 [  0.28740872]]

In [ ]:
spot_radius = 0.2  # Rspot/Rstar
n_spots = int(0.4/(spot_radius**2))
print(f'total coverage fraction is {n_spots*(spot_radius**2)}')
n_phases = 1

min_latitude = -80   # deg
max_latitude = 80   # deg
inc_stellar = 90 * u.deg
spot_lons,spot_lats,spot_radii,inc_stellar = generate_spots(min_latitude, max_latitude,
                                                spot_radius, n_spots,inclinations=inc_stellar)

# Star spot parameters for setting individual spots
# inc_stellar = 90 * u.deg
# spot_radii = np.array([[0.25], [0.1], [0.05],[0.05], [0.1], [0.05]])
# spot_lats = np.array([[-15], [50], [-20],[15], [0], [10]]) * u.deg
# spot_lons = np.array([[230], [200], [20],[180], [200], [120]]) * u.deg

plt.figure()
# plt.ylim(0.9,1.01)
for i in range(len(contrasts)):
    star = Star(spot_contrast=contrasts[i], u_ld=planet.u, rotation_period=4.863)
    rotation_lc = star.light_curve(spot_lons, spot_lats, spot_radii, inc_stellar,planet=noplanet, times=times.value)
    lcs[i,:] = rotation_lc[:,0]
    semi_amplitude = 100 * (np.amax(lcs[i,:])-np.amin(lcs[i,:]))/2
    plt.plot(lcs[i,:],label = semi_amplitude)
plt.legend(loc='upper right')
    
plt.figure()
ax = star.plot(spot_lons, spot_lats, spot_radii, inc_stellar, planet=planet,
               time=3)
plt.savefig('stellar_surface_map.png',dpi=100)

In [ ]:
generate_spots?

In [ ]:
for i in range(len(contrasts)):
    planet.u = [0.3,0.3]
    star = Star(spot_contrast=contrasts[i], u_ld=planet.u, rotation_period=4.863)
    rotation_lc = star.light_curve(spot_lons, spot_lats, spot_radii, inc_stellar,planet=noplanet, times=times.value)
    lcs[i,:] = rotation_lc[:,0]

r = RainbowWithModel(
    wavelength=w,
    time = times,
    # flux = lcs,
    flux=lcs*np.random.normal(1, 0.001, [N_wavelengths, N_times]),
    # uncertainty=np.ones([N_wavelengths, N_times]) * 0.0003,
    model=lcs)

plt.figure(figsize=(3,1))
ax = r.plot(spacing = 0.05)
# ax.set_ylim(0.8, 1.1)
plt.axvspan(0,0.417,label='10 hours',alpha=0.25)
plt.legend()
plt.savefig('rotation.png',dpi=100)

# Plot the chromatic rainbow imshow of this stellar rotation data
plt.figure(figsize=(6,4))
r.imshow(cmap='plasma')
plt.savefig('rotation_imshow.png',dpi=100)

# Plot the actual globe to see the whole star